##### Sprintの目的
- スクラッチを通して線形回帰を理解する
- オブジェクト指向を意識した実装に慣れる
- 数式をコードに落とし込めるようにする

スクラッチで線形回帰を実装した後、学習と検証を行なっていきます。



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

from decimal import Decimal, ROUND_HALF_UP

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

線形回帰のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


以下に雛形を用意してあります。このScratchLinearRegressionクラスにコードを書き加えていってください。

[(自分用メモ) 重回帰と勾配降下法](https://gneioagine.hatenablog.com/entry/20170304/1488588334)

In [109]:
#最新版
class ScratchLinearRegression():
    """
    線形回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ(重み)
    self.loss : 次の形のndarray, shape (self.iter,)
      訓練データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証データに対する損失の記録

    """
    def __init__(self, num_iter, lr=0.01, no_bias=True, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.no_bias = no_bias
        self.verbose = verbose
        # 損失を記録する配列を用意
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        線形回帰を学習する。検証データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        if self.no_bias == False:
            self.coef_ = np.random.rand(X.shape[1] + 1) #重み初期値：0以上1未満の乱数
        else:
            self.coef_ = np.random.rand(X.shape[1]) #重み初期値：0以上1未満の乱数
        
        for i in range(self.iter):
            #仮定関数の値（hypo_y）
            y_pred = self._linear_hypothesis(X)
            #誤差の算出 shape(m, 1)
            error = y_pred - y.reshape(-1, 1)
            #重みを更新
            self.coef_ = self._gradient_descent(X, error)
            print(self.coef_)
            #目的（損失）関数の値
            self.loss[i] = self.objective_func(X, error)
            
            #検証データがある場合のloss値保存
            if X_val is not None:
                self.val_loss = self.objective_func(X_val, y_val)
        
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print("train_loss:{}".format(self.loss))
            if X_val is not None:
                print("val_loss:{}".format(self.val_loss))
        
    def predict(self, X):
        """
        線形回帰を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """
        hypo_y = self._linear_hypothesis(X)
        
        return hypo_y
    
    def _linear_hypothesis(self, X):
        """
        線形の仮定関数を計算する

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          訓練データ

        Returns
        -------
        hypo_theta：次の形のndarray, shape (n_samples, 1) Q.この形にする意味は？？
         線形の仮定関数による推定結果
         予測値が返っている
        """
        if self.no_bias == False:
            #左列に1の列追加(切片用)
            one = np.ones(X.shape[0]).reshape(-1, 1)
            X = np.concatenate((one, X), axis=1)

        #θ：重み（初期値：0以上1未満の乱数）
        #self.coef_ = np.random.rand(X.shape[1])

        #線形回帰の仮定関数
        hypo_y = np.dot(X, self.coef_.reshape(-1, 1)) #(-1, 1)
    
        return hypo_y
    
    def _gradient_descent(self, X, error):
        """
        重みの更新
        Parameters
        --------
        X：次の形のndarray, shape (n_samples, n_features)
            訓練データ
        error：仮定関数の値　- 実際の値(n_samples, 1)

        Returns
        --------
        new_coef_：次の形のndarray, shape (1, n_features)
        """
        m = X.shape[0] #サンプル数
        
        if self.no_bias == False:
            one = np.ones(X.shape[0]).reshape(-1, 1)
            X = np.concatenate((one, X), axis=1)
            
        new_coef_ = self.coef_ - (self.lr*(1/m)*np.dot(error.reshape(-1, ), X)) #(coef数, )

        return new_coef_
    
    def objective_func(self, X, error):
        """ 
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          サンプル
        error：仮定関数の値　- 実際の値(n_samples, 1)

        Returns
        ----------
        loss : numpy.float
          平均二乗誤差/2
        """
        m = X.shape[0] #サンプルサイズ

        loss = (1/(2*m))*np.sum(np.square(error)) #float

        return loss

In [106]:
#編集前
class ScratchLinearRegression2222222222():
    """
    線形回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ(重み)
    self.loss : 次の形のndarray, shape (self.iter,)
      訓練データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証データに対する損失の記録

    """
    def __init__(self, num_iter, lr=0.01, no_bias=True, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.no_bias = no_bias
        self.verbose = verbose
        # 損失を記録する配列を用意
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        線形回帰を学習する。検証データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        self.coef_ = np.random.rand(X.shape[1] + 1) #重み初期値：0以上1未満の乱数
        
        for i in range(self.iter):
            #重みを更新
            self.coef_ = self._gradient_descent(X, y)
            print(self.coef_)
            #目的（損失）関数の値
            self.loss[i] = self.objective_func(X, y)
            
            #検証データがある場合のloss値保存
            if X_val is not None:
                self.val_loss = self.objective_func(X_val, y_val)
        
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print("train_loss:{}".format(self.loss))
            if X_val is not None:
                print("val_loss:{}".format(self.val_loss))
        
    def predict(self, X):
        """
        線形回帰を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """
        hypo_y, X = self._linear_hypothesis(X)
        
        return hypo_y
    
    def _linear_hypothesis(self, X):
        """
        線形の仮定関数を計算する

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          訓練データ

        Returns
        -------
        hypo_theta：次の形のndarray, shape (n_samples, 1) Q.この形にする意味は？？
         線形の仮定関数による推定結果
         予測値が返っている
        """
        #左列に1の列追加(切片用)
        one = np.ones(X.shape[0]).reshape(-1, 1)
        X_ = np.concatenate((one, X), axis=1)

        #θ：重み（初期値：0以上1未満の乱数）
        #self.coef_ = np.random.rand(X.shape[1])

        #線形回帰の仮定関数
        hypo_y = np.dot(X_, self.coef_.reshape(-1, 1)) #(-1, 1)
    
        return hypo_y, X_
    
    def _gradient_descent(self, X, y):
        """
        重みの更新
        Parameters
        --------
        X：次の形のndarray, shape (n_samples, n_features)
            訓練データ
        y：目的変数のndarray(n_features, )

        Returns
        --------
        new_coef_：次の形のndarray, shape (1, n_features)
        """
        hypo_y, X_ = self._linear_hypothesis(X)
        m = len(y) #サンプル数
        
        error = hypo_y - y.reshape(-1, 1) #誤差の算出 shape(m, 1)

        new_coef_ = self.coef_ - (self.lr*(1/m)*np.dot(error.reshape(-1, ), X_)) #(coef数, )

        return new_coef_
    
    def objective_func(self, X, y):
        """ 
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          サンプル
        y : 次の形のndarray, shape (n_samples,)

        Returns
        ----------
        loss : numpy.float
          平均二乗誤差/2
        """
        m = len(y) #サンプルサイズ
        y_pred, X_ = self._linear_hypothesis(X)

        loss = (1/(2*m))*np.sum(np.square(y_pred - y.reshape(-1, 1))) #float

        return loss

### 【問題1】仮定関数
以下の数式で表される線形回帰の仮定関数を実装してください。メソッドの雛形を用意してあります。

$$
h_θ(x) = θ_0x_0 + θ_1x_1 + ... + θ_jx_j + ... + θ_nx_n (x_0 = 1)
$$

x : 特徴量ベクトル

θ: パラメータベクトル

n: 特徴量の数

$x_j$ : j番目の特徴量

$θ_j$: j番目のパラメータ（重み）

特徴量の数 n は任意の値に対応できる実装にしてください。


なお、ベクトル形式で表すと以下のようになります。
$$
h_θ(x) = θ^Tx
$$


In [4]:
def _linear_hypothesis(self, X):
    """
    線形の仮定関数を計算する

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ

    Returns
    -------
    hypo_theta：次の形のndarray, shape (n_samples, 1) Q.この形にする意味は？？
     線形の仮定関数による推定結果
     予測値が返っている
    """
    #左列に1の列追加(切片用)
    one = np.ones(X.shape[0]).reshape(-1, 1)
    X = np.concatenate((one, X), axis=1)
    
    #θ：重み（初期値：0以上1未満の乱数）
    self.coef_ = np.random.rand(X.shape[1])
    
    #線形回帰の仮定関数
    hypo_y = np.dot(X, self.coef_.reshape(-1, 1)) #(-1, 1)
    
    return hypo_y, X

In [84]:
#確認用
def linear_hypothesis(X):
    """
    線形の仮定関数を計算する

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ

    Returns
    -------
    hypo_theta：次の形のndarray, shape (n_samples, 1) Q.この形にする意味は？？
     線形の仮定関数による推定結果
     予測値が返っている
    """
    #左列に1の列追加(切片用)
    one = np.ones(X.shape[0]).reshape(-1, 1)
    X = np.concatenate((one, X), axis=1)
    
    #θ：重み（初期値：0以上1未満の乱数）
    coef_ = np.random.rand(X.shape[1])
    
    #線形回帰の仮定関数
    hypo_y = np.dot(X, coef_.reshape(-1, 1)) #(-1, 1)
    
    return hypo_y, X

In [97]:
coef_ = np.random.rand(X.shape[1])
coef_

array([0.37018898, 0.0375793 ])

In [98]:
coef_ = np.random.rand(X.shape[1], 1)
coef_

array([[0.11191833],
       [0.04379995]])

In [99]:
coef_ = np.random.rand(X.shape[1]).reshape(-1, 1)
coef_

array([[0.2377156 ],
       [0.15746902]])

### 【問題2】最急降下法
最急降下法により学習させる実装を行なってください。  
以下の式で表されるパラメータの更新式のメソッドを追加し、fitメソッドから呼び出すようにしてください。

α: 学習率


i: サンプルのインデックス


j: 特徴量のインデックス

$$
θ_j := θ_j - α\frac{1}{m}\sum_{i=1}^{m}[(h_θ(x^{(i)}) - y^{(i)})x_j^{(i)}]
$$

In [ ]:
#最新版
def _gradient_descent(self, X, y):
    """
    「サンプル数、学習率、予測との誤差」から、新しい重みを算出
    Parameters
    --------
    X：次の形のndarray, shape (n_samples, n_features)
        訓練データ
    y：目的変数のndarray(n_features, )
    
    Returns
    --------
    new_coef_：次の形のndarray, shape (1, n_features)
    """
    hypo_y, X = self._linear_hypothesis(X)
    m = X.shape[0] #サンプル数
    
    error = hypo_y - y.reshape(-1, 1) #誤差の算出 shape(m, 1)
    
    new_coef_ = self.coef_ - self.lr*(1/m)*np.dot(error.reshape(-1,), X)
        
    return new_coef_

In [90]:
#不具合有り。これを用いると目的関数の値がどんどん大きくなってしまう。
def _gradient_descent(self, X, y):
    """
    「サンプル数、学習率、予測との誤差」から、新しい重みを算出
    Parameters
    --------
    X：次の形のndarray, shape (n_samples, n_features)
        訓練データ
    y：目的変数のndarray(n_features, )
    
    Returns
    --------
    new_coef_：次の形のndarray, shape (1, n_features)
    """
    hypo_y, X = self._linear_hypothesis(X)
    m = X.shape[0] #サンプル数
    
    error = hypo_y - y.reshape(-1, 1) #誤差の算出 shape(m, 1)
    
    new_coef_ = self.coef_ - self.lr*(1/m)*np.dot(error.reshape(-1,), X)
        
    return new_coef_

In [91]:
#確認用
def gradient_descent(X, y):
    """
    「サンプル数、学習率、予測との誤差」から、新しい重みを算出
    Parameters
    --------
    X：次の形のndarray, shape (n_samples, n_features)
        訓練データ
    y：目的変数のndarray(n_features, )
    
    Returns
    --------
    new_coef_：次の形のndarray, shape (1, n_features)
    """
    hypo_y, X = linear_hypothesis(X)
    m = X.shape[0] #サンプル数
    
    error = hypo_y - y.reshape(-1, 1) #誤差の算出 shape(m, 1)
    
    new_coef_ = coef_ - (0.03*(1/m)*np.dot(error.reshape(-1,), X))
        
    return new_coef_

### 【問題3】推定
推定する仕組みを実装してください。ScratchLinearRegressionクラスの雛形に含まれるpredictメソッドに書き加えてください。

仮定関数の出力が推定結果です。

### 【問題4】平均二乗誤差
線形回帰の指標値として用いられる平均二乗誤差（mean square error, MSE）の関数を作成してください。


平均二乗誤差関数は回帰問題全般で使える関数のため、ScratchLinearRegressionクラスのメソッドではなく、別の関数として作成してください。雛形を用意してあります。

平均二乗誤差は以下の数式で表されます。

$$
L(θ) = \frac{1}{m}\sum_{i=1}^{m}(h_θ(x^{(i)}) - y^{(i)})^2
$$

$m$ : 入力されるデータの数


$h_\theta()$ : 仮定関数


$x^{(i)}$ : i番目のサンプルの特徴量ベクトル


$y^{(i)}$ : i番目のサンプルの正解値

In [7]:
def MSE(y_pred, y):
    """
    平均二乗誤差の計算

    Parameters
    ----------
    y_pred : 次の形のndarray, shape (n_samples,)
      推定した値
    y : 次の形のndarray, shape (n_samples,)
      正解値

    Returns
    ----------
    mse : numpy.float
      平均二乗誤差
    """
    m = len(y) #サンプル数
    mse = (1/m)*np.sum(np.square(y_pred - y)) #float
    
    return mse

### 【問題5】目的関数
以下の数式で表される線形回帰の 目的関数（損失関数） を実装してください。そして、これをself.loss, self.val_lossに記録するようにしてください。

$$
J(θ) = \frac{1}{2m}\sum_{i=1}^{m}(h_θ(x^{(i)}) - y^{(i)})^2
$$

$m$ : 入力されるデータの数


$h_\theta()$ : 仮定関数


$x^{(i)}$ : i番目のサンプルの特徴量ベクトル


$y^{(i)}$ : i番目のサンプルの正解値

In [22]:
def objective_func(X, y):
    """
    
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      サンプル
    y : 次の形のndarray, shape (n_samples,)
    
    Returns
    ----------
    loss : numpy.float
      平均二乗誤差/2
    """
    m = len(y) #サンプルサイズ
    y_pred, X = self._linear_hypothesis(X)
    
    mse = MSE(y_pred, y.reshape(-1, 1)) #平均二乗誤差の計算
    
    loss = mse/2 #float
    
    return loss

In [9]:
#確認用
def objective_func2(X, y):
    """
    
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      サンプル
    y : 次の形のndarray, shape (n_samples,)
    
    Returns
    ----------
    loss : numpy.float
      平均二乗誤差/2
    """
    m = len(y) #サンプルサイズ
    y_pred, X = linear_hypothesis(X)
    
    loss = 1/(2*m)*np.sum(np.square(y_pred - y.reshape(-1, 1))) #float
    
    return loss

In [10]:
P = np.array([2, 3]).reshape(-1, 1)
P

array([[2],
       [3]])

In [11]:
np.square(P)

array([[4],
       [9]])

In [12]:
np.sum(np.square(P))

13

### 【問題6】学習と推定
機械学習スクラッチ入門のSprintで用意したHouse Pricesコンペティションのデータに対してスクラッチ実装の学習と推定を行なってください。


scikit-learnによる実装と比べ、正しく動いているかを確認してください。

In [13]:
df = pd.read_csv("train.csv")
X = df.loc[:, ["GrLivArea", "YearBuilt"]].values
y = df.loc[:, "SalePrice"].values

In [14]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [39]:
display(X)
display(y)

array([[1710, 2003],
       [1262, 1976],
       [1786, 2001],
       ...,
       [2340, 1941],
       [1078, 1950],
       [1256, 1965]])

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

model = ScratchLinearRegression(num_iter=10, lr=0.03, no_bias=False, verbose=True)

model.fit(X_train, y_train)

[5.41376667e+03 9.09869551e+06 1.07101017e+07]
[-1.04807218e+09 -1.66646786e+12 -2.06709774e+12]
[1.98198498e+14 3.14711888e+17 3.90899931e+17]
[-3.74610143e+19 -5.94808362e+22 -7.38830217e+22]
[7.08031844e+24 1.12421644e+28 1.39642593e+28]
[-1.33821501e+30 -2.12482431e+33 -2.63931368e+33]
[2.52929216e+35 4.01602241e+38 4.98843263e+38]
[-4.78048654e+40 -7.59047981e+43 -9.42838293e+43]
[9.03535460e+45 1.43463800e+49 1.78201073e+49]
[-1.70772644e+51 -2.71153636e+54 -3.36808790e+54]
train_loss:[1.94514908e+010 6.21691925e+020 2.22082145e+031 7.93340894e+041
 2.83404045e+052 1.01240026e+063 3.61658312e+073 1.29194687e+084
 4.61520353e+094 1.64868263e+105]


/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [70]:
aa = np.array([1, 2]).reshape(-1, 1)
aa

array([[1],
       [2]])

In [72]:
bb = np.array([1, 2, 3, 4, 5, 6]).reshape(2, 3)
bb

array([[1, 2, 3],
       [4, 5, 6]])

In [73]:
aa*bb

array([[ 1,  2,  3],
       [ 8, 10, 12]])

In [80]:
X

array([[1710, 2003],
       [1262, 1976],
       [1786, 2001],
       ...,
       [2340, 1941],
       [1078, 1950],
       [1256, 1965]])

In [81]:
X_train

array([[1530, 2005],
       [1922, 2003],
       [1442, 1958],
       ...,
       [1576, 2006],
       [1362, 1993],
       [1983, 1999]])